# Env/Program Setup

In [ ]:
%tensorflow_version 1.x

In [ ]:
!git clone https://github.com/whornsby/neural-style.git
!mv neural-style/* .
!rm -rf neural-style/

In [ ]:
!pip install -r requirements.txt

In [ ]:
!curl 'https://www.vlfeat.org/matconvnet/models/imagenet-vgg-verydeep-19.mat' --output imagenet-vgg-verydeep-19.mat

# Utils and Project Code

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from util import *
from transfer import *

## Image Utils

In [ ]:
import os

def process_upload(img_data, content=False, style=False):
  filename = list(img_data)[0]
  temp_fn = filename.strip().replace(" ", "_")

  if filename != temp_fn:
    os.rename(filename, temp_fn)
    filename = temp_fn
  
  if content and style:
    raise AttributeError("Image must be either content or style, not both")
  if content:
    CONTENT_IMAGE_FN = filename
    print("Content image filename:", filename)
    imgfig(filename, "Content image")
  elif style:
    STYLE_IMAGE_FN = filename
    print("Style image filename:", filename)
    imgfig(filename, "Style image")
  else:
    print("Image saved with filename:", filename)
    imgshow(filename)

## Jobs

In [ ]:
transfer_jobs = []
outputs = []


def run_jobs():
  for job in transfer_jobs:
    output_name = job.Output_name()
    print("Generating "+output_name+"...")
    job.Execute()
    outputs.append(output_name)
    imgfig(output, "Style Transferred Image - " + output, 20)

## Drive

In [ ]:
ROOT_DIR_GDID = '13kelXjrToB3IpbHNj0WmRtP0yRNHcNUg'
CONTENT_DIR_GDID = '11_SKmzv0UqQhxb-rgcLYVsF5sdfUlu6A'
STYLE_DIR_GDID = '1ExIrZyGT2u4brgBIqVzQQk6-j63XP1xu'
OUTPUT_DIR_GDID = '190RRdQlpmipXHztkyK1tF_6WTeA5Ulmo'

'''Recursively search through Drive to find the file with a given name''' 
def get_file_from_name(name, start_id='root'):
  parents = [start_id]
  while parents:
    pid = parents.pop()
    listed = drive.ListFile({'q': f"'{pid}' in parents"}).GetList()
    for file in listed:
      if file['title'] == name:
        pass 
      # return file
      elif file['mimeType'] == 'application/vnd.google-apps.folder':
          parents.append(file['id'])

def get_file_by_id(id,name):
  file = drive.CreateFile({'id': id})
  file.GetContentFile('tmp/' + name)

def upload_image(im_file, parent_folder, title=None):
  if title is None:
    title = im_file.split('/')[-1]

  upload_file = drive.CreateFile({'title': title, 'parents': [{'id': parent_folder}]})
  upload_file.SetContentFile(im_file)
  upload_file.Upload()


def download_image(filename, parent_id):
  file_list = drive.ListFile({'q': f'{parent_id} in parents and trashed=false'})



# Manual Upload

In [ ]:
#@title Image Overrides
manual_content_image = "" #@param {type:"string"}
enable_content_override = True #@param {type:"boolean"}
manual_style_image = "" #@param {type:"string"}
enable_style_override = True #@param {type:"boolean"}

if enable_content_override: CONTENT_IMAGE_FN = manual_content_image
if enable_style_override: STYLE_IMAGE_FN = manual_style_image

imgshow_h(CONTENT_IMAGE_FN, STYLE_IMAGE_FN)

In [ ]:
from google.colab import files

content_img = files.upload()
process_upload(content_img, content=True)
# CONTENT_IMAGE_FN = list(content_img)[0]

In [ ]:
style_img = files.upload()
process_upload(style_img, style=True)
# STYLE_IMAGE_FN = list(style_img)[0]

# Style Transfer

In [ ]:
#@title  { vertical-output: true }
#@markdown #Command Parameters 
content_image = "examples/1-content.jpg" #@param {type:"string"}
style_image = "examples/1-style.jpg" #@param {type:"string"}
output_name = "example1" #@param {type:"string"}
output_width = 1000 #@param {type:"slider", min:500, max:1920, step:10}

#@markdown ---
#@markdown ## Processing

iterations = 1000 #@param {type:"slider", min:500, max:2000, step:100}

# Either 'max' or 'avg' - Default: 'max'
# Original VGG topology uses max pooling, but paper suggests replacing it with avg.
# The outputs are perceptually different, max in general tends to have finer detail style transfer, but could have troubles at lower-freqency detail level
pooling = "max" #@param ["max", "avg"]
preserve_colors = False #@param {type: "boolean"}

#@markdown #### Logging
# keep_progress_logs = False #@param {type:"boolean"}
# print_iterations =   0 #@param {type:"integer"}
# checkpoint_iterations = 200 #@param {type:"integer"}
# checkpoint_output = False #@param {type:"boolean"}


#@markdown ---

#TODO: checkpoint_output and checkpoint_iterations
#TODO: style blending

#@markdown ## Tuning
#@markdown ### __Learning__
#@markdown Weights are used in the cost function to balance the similarites of the content image and style image(s) to the generated image.
#@markdown Learning rate is how quickly adjustments are made to lower the cost function.

# content_weight_scale = 1 #@param {type:"slider", min:0.1, max:5, step:0.1}
# style_weight_scale = 1 #@param {type:"slider", min:0.1, max:5, step:0.1}
# learning_rate = 1 #@param {type:"slider", min:0.1, max:2, step:0.1}


# #@markdown ### __Style Abstractness__
# #@markdown _Float greater than 0.0 - Default: 1.0 (STYLE_LAYER_WEIGHT_EXP)_
# #@markdown * Lower values mean __finer details__ of the style will be transfered (also preserves more details of the content image)
# #@markdown * Higher values favor the __coarser/larger features__ of the style (can make output look more artsy since the details of the content image can not be represented by the larger stylistic features)
# style_abstractness = 0.5 #@param {type:"slider", min:0.0, max:5.0, step:0.1}
# style_abstractness_enabled = False #@param {type:"boolean"}

# #@markdown ### Content Abstractness
# #@markdown _Float between 0.0 and 1.0 - Default: 1.0 (CONTENT_WEIGHT_BLEND)_

# #@markdown Conceptually similar to the above, but pertains the the detail/abstracness of the content image
# #@markdown * Lower values make the output __MORE abstract__ (reverse of above)
# content_abstractness = 1 #@param {type:"slider", min:0.0, max:1.0, step:0.1} 
# content_abstractness_enabled = False #@param {type:"boolean"}



# CONTENT_IMAGE_FN = content_image
# STYLE_IMAGE_FN = style_image
# # Script Variables
# OUTPUT_WIDTH = output_width # min(output_width, Image.open(CONTENT_IMAGE_FN).size[0])
# POOLING = pooling if pooling in ['max', 'avg'] else 'max'

# CONTENT_WEIGHT = 5e0 * content_weight_scale  # pulled from neural-style defaults
# STYLE_WEIGHT = 5e2 * style_weight_scale  # pulled from neural-style defaults
# LEARNING_RATE = learning_rate

# STYLE_LAYER_WEIGHT_EXP = style_abstractness if style_abstractness_enabled and 0 < style_abstractness else 1.0
# CONTENT_WEIGHT_BLEND = content_abstractness if content_abstractness_enabled and 0 < content_abstractness < 1 else 1.0

IS_TEST = True #@param {type:"boolean"}
if IS_TEST:
  output_width = 360
  iterations = 50
  output_name = "TEST_" + output_name
  
job = TransferJob(content_image, style_image, output_name, output_width)

job.iterations(iterations)
if preserve_colors:
  job.preserve_colors()
job.pooling(pooling)
job.overwrite()

# if keep_progress_logs: 
#   flags.append("--progress-write")
# if print_iterations > 0:
#   flags.append(f"--print-iterations {print_iterations}")
# if checkpoint_output:
#   flags.append(f"--checkpoint-output {output_filename}_%05d.jpg")
#   flags.append(f"--checkpoint-iterations {checkpoint_iterations}")

# if content_weight_scale != 1:
#   flags.append(f"--content-weight {CONTENT_WEIGHT}")
#   param_string += f"_cWgt{content_weight_scale}"
# if  style_weight_scale != 1:
#   flags.append(f"--style-weight {STYLE_WEIGHT}")
#   param_string += f"_sWgt{style_weight_scale}"
# if learning_rate != 1:
#   flags.append(f"--learning-rate {LEARNING_RATE}")
#   param_string += f"_lr{learning_rate}"

# if style_abstractness_enabled and style_abstractness != 1: 
#   flags.append(f"--style-layer-weight-exp {STYLE_LAYER_WEIGHT_EXP}")
#   param_string += f"_sAb{STYLE_LAYER_WEIGHT_EXP}"
# if content_abstractness_enabled and content_abstractness != 1: 
#   flags.append(f"--content-weight-blend {CONTENT_WEIGHT_BLEND}")
#   param_string += f"_cAb{CONTENT_WEIGHT_BLEND}"

TRANSFER_JOBS_LIST = "RESET" #@param ["RESET", "APPEND"]

if TRANSFER_JOBS_LIST is "RESET":
  transfer_jobs = [job]
  print(output_name,"-",job.Output_name())
elif TRANSFER_JOBS_LIST is "APPEND":
  transfer_jobs.append(job)
  print(output_name,"-",job.Output_name())

print("\nJobs to run:")
for j in transfer_jobs:
  print("  ",j.Output_name())


In [ ]:
run_jobs()

In [ ]:
for out in outputs:
  imgfig(out, "Style Transferred Image - " + out, 20)